In [2]:
import numpy as np

# -------------------------------------------------------------
# 1. Representasi embedding sederhana dari 8 token
# -------------------------------------------------------------
X = np.array([
    [ 1,  0,  1,  0, -1,  0],   # gimana
    [ 0,  1,  0, -1,  1,  0],   # nih
    [ 1, -1,  0,  0,  0,  1],   # apa
    [ 0,  0, -1,  1,  0,  1],   # gak
    [-1,  1,  0,  1,  0,  0],   # ken
    [ 1,  0, -1,  0,  1,  0],   # update
    [ 0,  1,  1,  0,  0, -1],   # kai
    [ 1,  0,  0,  1, -1,  0]    # access
])

# -------------------------------------------------------------
# 2. Matriks bobot proyeksi 0Query, Key, dan Value
#    (6 dimensi → 3 dimensi)
# -------------------------------------------------------------
WQ = np.array([
    [ 1,  0,  1],
    [ 0,  1, -1],
    [ 1, -1,  0],
    [ 0,  1,  0],
    [-1,  0,  1],
    [ 1,  0, -1]
])

WK = np.array([
    [ 0,  1,  0],
    [ 1, -1,  0],
    [-1,  0,  1],
    [ 1,  0, -1],
    [ 0,  1,  1],
    [ 1,  0,  0]
])

WV = np.array([
    [ 1,  0,  0],
    [ 0,  1,  0],
    [ 1,  1, -1],
    [ 0,  0,  1],
    [ 1,  0,  1],
    [ 0, -1,  1]
])

# -------------------------------------------------------------
# 3. Menghitung Q, K, V
# -------------------------------------------------------------

In [3]:
Q = X @ WQ
Q

array([[ 3, -1,  0],
       [-1,  0,  0],
       [ 2, -1,  1],
       [ 0,  2, -1],
       [-1,  2, -2],
       [-1,  1,  2],
       [ 0,  0,  0],
       [ 2,  1,  0]])

In [4]:
K = X @ WK
K

array([[-1,  0,  0],
       [ 0,  0,  2],
       [ 0,  2,  0],
       [ 3,  0, -2],
       [ 2, -2, -1],
       [ 1,  2,  0],
       [-1, -1,  1],
       [ 1,  0, -2]])

In [5]:
V = X @ WV
V

array([[ 1,  1, -2],
       [ 1,  1,  0],
       [ 1, -2,  1],
       [-1, -2,  3],
       [-1,  1,  1],
       [ 1, -1,  2],
       [ 1,  3, -2],
       [ 0,  0,  0]])

In [6]:
K.T

array([[-1,  0,  0,  3,  2,  1, -1,  1],
       [ 0,  0,  2,  0, -2,  2, -1,  0],
       [ 0,  2,  0, -2, -1,  0,  1, -2]])

In [7]:
# Qkt = Q @ K.T
# Qkt
# -------------------------------------------------------------
# 4. Attention Score (QKᵀ)
# -------------------------------------------------------------
attention_scores1 = Q @ K.T
attention_scores1

array([[-3,  0, -2,  9,  8,  1, -2,  3],
       [ 1,  0,  0, -3, -2, -1,  1, -1],
       [-2,  2, -2,  4,  5,  0,  0,  0],
       [ 0, -2,  4,  2, -3,  4, -3,  2],
       [ 1, -4,  4,  1, -4,  3, -3,  3],
       [ 1,  4,  2, -7, -6,  1,  2, -5],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [-2,  0,  2,  6,  2,  4, -3,  2]])

In [8]:
# # -------------------------------------------------------------
# # 5. Softmax untuk bobot perhatian
# # -------------------------------------------------------------
# # def softmax(x):
# #     e = np.exp(x - np.max(x))
# #     return e / e.sum(axis=-1, keepdims=True)
# def softmax(x):
#     # stabilitas numerik: kurangi nilai maksimum di tiap baris
#     e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
#     return e_x / np.sum(e_x, axis=1, keepdims=True)

# attention_weights1 = softmax(attention_scores1)

# # -------------------------------------------------------------
# # 6. Attention Output
# # -------------------------------------------------------------
# attention_output1 = attention_weights1 @ V

# print("\nAttention Scores:\n", attention_scores1)
# print("\nAttention Weights:\n", attention_weights1)
# print("\nContextual Embedding (Attention Output):\n", attention_output1)


In [9]:
d_k = 3
sqrt_dk = np.sqrt(d_k)
scaled_attention_logits = attention_scores1 / sqrt_dk
scaled_attention_logits

array([[-1.73205081,  0.        , -1.15470054,  5.19615242,  4.61880215,
         0.57735027, -1.15470054,  1.73205081],
       [ 0.57735027,  0.        ,  0.        , -1.73205081, -1.15470054,
        -0.57735027,  0.57735027, -0.57735027],
       [-1.15470054,  1.15470054, -1.15470054,  2.30940108,  2.88675135,
         0.        ,  0.        ,  0.        ],
       [ 0.        , -1.15470054,  2.30940108,  1.15470054, -1.73205081,
         2.30940108, -1.73205081,  1.15470054],
       [ 0.57735027, -2.30940108,  2.30940108,  0.57735027, -2.30940108,
         1.73205081, -1.73205081,  1.73205081],
       [ 0.57735027,  2.30940108,  1.15470054, -4.04145188, -3.46410162,
         0.57735027,  1.15470054, -2.88675135],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [-1.15470054,  0.        ,  1.15470054,  3.46410162,  1.15470054,
         2.30940108, -1.73205081,  1.15470054]])

In [10]:
def softmax(x):
    # stabilitas numerik: kurangi nilai maksimum di tiap baris
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / np.sum(e_x, axis=1, keepdims=True)

# def softmax1(x):
#     e = np.exp(x - np.max(x))
#     return e / e.sum(axis=-1, keepdims=True)

attention_weights2 = softmax(scaled_attention_logits)
attention_weights2

array([[6.07581153e-04, 3.43419065e-03, 1.08229171e-03, 6.20132834e-01,
        3.48132597e-01, 6.11736562e-03, 1.08229171e-03, 1.94108481e-02],
       [2.48181232e-01, 1.39324951e-01, 1.39324951e-01, 2.46495384e-02,
        4.39085229e-02, 7.82147864e-02, 2.48181232e-01, 7.82147864e-02],
       [9.05434064e-03, 9.11626568e-02, 9.05434064e-03, 2.89265770e-01,
        5.15272638e-01, 2.87300844e-02, 2.87300844e-02, 2.87300844e-02],
       [3.55215487e-02, 1.11946835e-02, 3.57644900e-01, 1.12712469e-01,
        6.28451524e-03, 3.57644900e-01, 6.28451524e-03, 1.12712469e-01],
       [7.03768664e-02, 3.92400675e-03, 3.97786494e-01, 7.03768664e-02,
        3.92400675e-03, 2.23310939e-01, 6.98988099e-03, 2.23310939e-01],
       [8.87028441e-02, 5.01369202e-01, 1.58007456e-01, 8.75018543e-04,
        1.55868118e-03, 8.87028441e-02, 1.58007456e-01, 2.77649777e-03],
       [1.25000000e-01, 1.25000000e-01, 1.25000000e-01, 1.25000000e-01,
        1.25000000e-01, 1.25000000e-01, 1.25000000e-01, 1.

In [11]:
np.set_printoptions(formatter={'float': lambda x: f"{x:0.3f}"})

print(attention_weights2)

[[0.001 0.003 0.001 0.620 0.348 0.006 0.001 0.019]
 [0.248 0.139 0.139 0.025 0.044 0.078 0.248 0.078]
 [0.009 0.091 0.009 0.289 0.515 0.029 0.029 0.029]
 [0.036 0.011 0.358 0.113 0.006 0.358 0.006 0.113]
 [0.070 0.004 0.398 0.070 0.004 0.223 0.007 0.223]
 [0.089 0.501 0.158 0.001 0.002 0.089 0.158 0.003]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
 [0.006 0.019 0.060 0.602 0.060 0.190 0.003 0.060]]


In [12]:
attention_output = attention_weights2 @ V
attention_output

array([[-0.956, -0.893, 2.218],
       [0.785, 0.770, -0.579],
       [-0.638, 0.076, 1.374],
       [0.649, -1.227, 1.334],
       [0.628, -1.060, 0.905],
       [0.992, 0.659, -0.154],
       [0.375, 0.125, 0.375],
       [-0.384, -1.420, 2.288]])

In [13]:
np.random.seed(42)
wq2, wk2, wv2 = [np.random.randint(-1, 2, (9, 3)) for _ in range(3)]
wq3, wk3, wv3 = [np.random.randint(-1, 2, (9, 3)) for _ in range(3)]

In [14]:
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

def attention(X, Wq, Wk, Wv):
    d_k = Wq.shape[1]
    Q, K, V = X @ Wq, X @ Wk, X @ Wv
    scores = (Q @ K.T) / np.sqrt(d_k)
    weights = softmax(scores)
    return weights @ V

In [15]:
head2 = attention(X, wq2, wk2, wv2)
head3 = attention(X, wq3, wk3, wv3)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 9 is different from 6)

In [16]:
head2

NameError: name 'head2' is not defined

In [19]:
head3

array([[-2.716, 2.349, 1.096],
       [-2.860, 1.814, 0.886],
       [-3.632, 2.502, 1.549],
       [-1.412, 2.551, 1.983],
       [-1.058, 2.012, 0.035],
       [-1.731, 1.885, -0.321],
       [-1.441, 2.192, 0.572],
       [-3.571, 0.524, -2.125]])

In [20]:
multi_head_concat = np.concatenate([attention_output, head2, head3], axis=1)
multi_head_concat

array([[1.167, -0.426, 1.981, 0.318, -0.560, -1.352, -2.716, 2.349,
        1.096],
       [2.249, -0.971, 1.100, -0.404, -0.818, -0.823, -2.860, 1.814,
        0.886],
       [2.249, -0.971, 1.100, -0.966, -1.932, -0.969, -3.632, 2.502,
        1.549],
       [-0.796, -1.880, 5.464, 0.009, -0.984, -0.997, -1.412, 2.551,
        1.983],
       [-0.085, -0.981, 2.796, 0.019, 0.851, -1.920, -1.058, 2.012,
        0.035],
       [2.365, -1.072, 1.335, -0.111, 0.171, -0.968, -1.731, 1.885,
        -0.321],
       [1.000, -1.000, 1.206, 0.480, -0.484, -1.483, -1.441, 2.192,
        0.572],
       [0.579, -1.232, 2.166, -0.001, 0.897, -1.815, -3.571, 0.524,
        -2.125]])

In [17]:
W_o = np.random.randint(-1, 2, (9, 9))
W_o

array([[ 0,  0,  0, -1,  0, -1, -1,  0,  0],
       [ 0,  1,  0,  1, -1,  1,  0, -1, -1],
       [-1,  1,  0, -1, -1, -1,  1,  1,  0],
       [ 1, -1,  0, -1, -1,  1,  0,  1,  1],
       [ 0, -1, -1,  0, -1,  0,  0,  1,  0],
       [ 1, -1, -1, -1, -1,  1, -1,  0,  0],
       [ 0,  1, -1, -1, -1,  1,  0,  0, -1],
       [ 0,  0,  1,  1,  1,  1, -1,  1,  0],
       [-1,  0,  0,  0,  1,  1, -1, -1,  1]], dtype=int32)

In [22]:
attention_output_final = multi_head_concat @ W_o
attention_output_final

array([[-4.110, 0.433, 6.976, 2.525, 6.198, -3.877, -1.279, 3.419, 4.555],
       [-3.214, -0.685, 6.315, 1.581, 7.476, -5.708, -3.026, 1.778,
        4.312],
       [-4.585, 0.365, 9.035, 3.748, 11.421, -5.836, -4.231, 0.126,
        5.186],
       [-8.436, 4.145, 5.943, -1.598, 4.334, -4.414, 2.723, 6.935, 5.283],
       [-4.732, 1.807, 4.138, 1.279, 2.339, -4.605, 2.754, 6.623, 2.092],
       [-2.094, -0.559, 4.414, -0.076, 3.941, -6.018, -1.626, 4.672,
        2.371],
       [-2.782, 0.252, 5.600, 1.431, 5.486, -2.888, -1.074, 3.822, 3.493],
       [-1.857, -1.718, 5.012, 1.934, 1.954, -10.964, 5.003, 6.943,
        2.677]])

In [23]:
def xavier_uniform(shape, rng=np.random):
    fan_in, fan_out = shape[0], shape[1]
    limit = np.sqrt(6.0/(fan_in + fan_out))
    return rng.uniform(-limit, limit, size=shape)

W_o_xavier = xavier_uniform((9, 9), np.random)
W_o_xavier


array([[-0.362, -0.530, 0.105, 0.205, -0.558, 0.014, -0.316, 0.168,
        -0.376],
       [0.220, -0.131, 0.504, -0.419, -0.184, -0.446, 0.490, 0.436,
        -0.280],
       [0.185, 0.366, 0.064, 0.034, -0.298, -0.470, 0.459, 0.462, 0.154],
       [-0.186, -0.174, 0.261, 0.459, 0.447, 0.323, 0.164, -0.480,
        -0.391],
       [0.460, 0.123, -0.567, -0.460, 0.189, -0.572, -0.392, 0.056,
        0.222],
       [0.175, -0.318, 0.245, -0.303, -0.202, 0.285, 0.173, 0.403, 0.182],
       [0.079, -0.469, -0.153, -0.271, -0.296, 0.546, -0.123, 0.453,
        0.151],
       [0.340, 0.003, 0.089, -0.009, -0.352, 0.257, -0.253, -0.549,
        0.168],
       [-0.373, 0.509, 0.524, 0.479, -0.150, -0.560, 0.495, -0.083,
        0.539]])

In [24]:
attentioin_final_xavier = multi_head_concat @ W_o_xavier
attentioin_final_xavier

array([[-0.527, 2.307, 1.301, 2.540, -1.043, -2.179, 0.652, -2.414,
        0.064],
       [-1.209, 1.367, 1.035, 2.530, -1.500, -1.542, -0.238, -2.086,
        -0.229],
       [-1.717, 2.076, 2.011, 3.351, -2.046, -1.744, 0.330, -2.720,
        0.073],
       [0.271, 4.543, 1.114, 2.881, -1.598, -2.683, 2.560, -1.092, 2.545],
       [0.971, 2.431, -0.914, 0.974, -0.450, -1.985, -0.195, -1.471,
        0.764],
       [-0.292, 0.379, -0.306, 1.443, -1.447, -0.807, -1.334, -1.573,
        -0.598],
       [-0.513, 1.345, 0.428, 2.203, -0.743, -0.642, -0.335, -2.470,
        -0.018],
       [0.703, 1.932, -1.898, 0.791, 0.982, -2.116, -1.202, -1.847,
        -1.269]])

In [26]:
attentioin_final_xavier.shape

(8, 9)

In [18]:
def layer_norm(x, eps=1e-5):
    # normalisasi per token (per baris)
    mu  = x.mean(axis=1, keepdims=True)
    var = x.var(axis=1, keepdims=True)
    return (x - mu) / np.sqrt(var + eps), mu.squeeze(), np.sqrt(var + eps).squeeze()

residual = V + attention_output
residual

array([[0.044, 0.107, 0.218],
       [1.785, 1.770, -0.579],
       [0.362, -1.924, 2.374],
       [-0.351, -3.227, 4.334],
       [-0.372, -0.060, 1.905],
       [1.992, -0.341, 1.846],
       [1.375, 3.125, -1.625],
       [-0.384, -1.420, 2.288]])

In [19]:
normed, row_mean, row_std = layer_norm(residual)
normed

array([[-1.095, -0.225, 1.321],
       [0.714, 0.700, -1.414],
       [0.052, -1.250, 1.198],
       [-0.193, -1.116, 1.310],
       [-0.856, -0.547, 1.403],
       [0.774, -1.412, 0.638],
       [0.212, 1.105, -1.317],
       [-0.349, -1.012, 1.361]])

In [20]:
row_mean

array([0.123, 0.992, 0.271, 0.252, 0.491, 1.166, 0.958, 0.161])

In [21]:
row_std

array([0.072, 1.111, 1.756, 3.116, 1.008, 1.067, 1.961, 1.562])

## FFNN

In [ ]:
# import numpy as np
# def xavier_uniform(shape, rng=np.random):
#     fan_in, fan_out = shape[0], shape[1]
#     limit = np.sqrt(6.0 / (fan_in + fan_out))
#     return rng.uniform(-limit, limit, size=shape)

# # === FFNN layer 1 ===
# def ffnn_layer1(X, out_dim=18, bias_value=1.0, seed=42):
#     """
#     X         : numpy array (N, D_in) -> input hasil LayerNorm
#     out_dim   : jumlah neuron layer FF pertama (default 18)
#     bias_value: nilai bias (default 1)
#     seed      : agar hasil acak reproducible
#     """
#     np.random.seed(seed)

#     N, D_in = X.shape  # misal (8, 9)
#     D_out = out_dim    # misal 18

#     # Inisialisasi bobot dan bias
#     W1 = xavier_uniform((D_in, D_out))
#     b1 = np.full((D_out,), bias_value, dtype=float)

#     # Linear transform (belum ReLU)
#     Z = np.dot(X, W1) + b1  # (8×9) · (9×18) + (1×18) -> (8×18)

#     return Z, W1, b1

In [22]:
import numpy as np

# === He Initialization (Kaiming) ===
def he_initialization(shape, rng=np.random):
    """
    shape: (fan_in, fan_out)
    menggunakan distribusi normal std = sqrt(2/fan_in)
    """
    fan_in = shape[0]
    std = np.sqrt(2.0 / fan_in)
    return rng.normal(0.0, std, size=shape)

# === FFNN Layer 1 ===
def ffnn_layer1(X, out_dim=18, bias_value=1.0, seed=42):
    """
    X         : numpy array (N, D_in) hasil LayerNorm
    out_dim   : jumlah neuron hidden layer FF pertama
    bias_value: nilai bias (bias default = 1)
    seed      : untuk reproducibility
    """
    np.random.seed(seed)

    N, D_in = X.shape
    D_out = out_dim

    # Bobot dari He Initialization
    W1 = he_initialization((D_in, D_out))
    b1 = np.full((D_out,), bias_value, dtype=float)

    # Linear transform
    Z = np.dot(X, W1) + b1

    return Z, W1, b1


In [23]:
np.set_printoptions(formatter={'float': lambda x: f"{x:0.3f}"})

In [24]:
layer_norm_x = np.array([
    [-1.095, -0.225, 1.320],
    [ 0.713,  0.701, -1.415],
    [ 0.052, -1.249,  1.198],
    [-0.193, -1.116,  1.310],
    [-0.855, -0.547,  1.403],
    [ 0.774, -1.412,  0.638],
    [ 0.212,  1.104, -1.316],
    [-0.349, -1.012,  1.362]
], dtype=float)

# Jalankan layer FFNN pertama
Z, W1, b1 = ffnn_layer1(layer_norm_x, out_dim=18, bias_value=1.0, seed=42)

print("Z shape:", Z.shape)
print("W1 shape:", W1.shape)
print("W1:\n", np.round(W1, 4))
print("b1 shape:", b1.shape)
print("\nPreview output Z (belum ReLU):\n", np.round(Z, 4))

Z shape: (8, 18)
W1 shape: (3, 18)
W1:
 [[0.406 -0.113 0.529 1.244 -0.191 -0.191 1.289 0.627 -0.383 0.443 -0.378
  -0.380 0.198 -1.562 -1.408 -0.459 -0.827 0.257]
 [-0.741 -1.153 1.197 -0.184 0.055 -1.163 -0.445 0.091 -0.940 0.307
  -0.490 -0.238 -0.491 1.512 -0.011 -0.864 0.672 -0.997]
 [0.171 -1.600 -1.085 0.161 0.603 0.140 -0.094 -0.246 -1.207 -0.588
  -0.376 0.863 0.281 -1.440 0.265 -0.314 -0.553 0.499]]
b1 shape: (18,)

Preview output Z (belum ReLU):
 [[0.948 -0.729 -1.280 -0.108 1.993 1.656 -0.437 -0.031 0.038 -0.330 1.028
  2.609 1.265 0.470 2.894 1.282 1.025 1.603]
 [0.528 2.375 3.751 1.530 0.049 -0.150 1.741 1.858 1.776 2.363 0.919
  -0.659 0.400 2.983 -0.386 0.512 1.663 -0.223]
 [2.151 0.517 -1.766 1.488 1.643 2.611 1.509 0.625 0.708 -0.064 1.142
  2.312 1.960 -2.695 1.258 1.678 -0.544 2.857]
 [1.972 0.213 -1.858 1.176 1.765 2.518 1.123 0.456 0.541 -0.198 1.128
  2.470 1.878 -2.272 1.631 1.641 -0.314 2.717]
 [1.298 -0.518 -1.628 0.263 1.979 1.996 0.008 0.070 0.148 -0.371 1.06

In [27]:
W1

array([[0.406, -0.113, 0.529, 1.244, -0.191, -0.191, 1.289, 0.627,
        -0.383, 0.443, -0.378, -0.380, 0.198, -1.562, -1.408, -0.459,
        -0.827, 0.257],
       [-0.741, -1.153, 1.197, -0.184, 0.055, -1.163, -0.444, 0.091,
        -0.940, 0.307, -0.490, -0.238, -0.491, 1.512, -0.011, -0.864,
        0.672, -0.997],
       [0.171, -1.600, -1.084, 0.161, 0.603, 0.140, -0.094, -0.246,
        -1.207, -0.588, -0.376, 0.863, 0.281, -1.440, 0.265, -0.314,
        -0.553, 0.499]])

In [26]:
Z

array([[0.948, -0.729, -1.280, -0.108, 1.993, 1.656, -0.437, -0.031,
        0.038, -0.330, 1.028, 2.609, 1.265, 0.470, 2.894, 1.282, 1.025,
        1.603],
       [0.528, 2.375, 3.750, 1.530, 0.049, -0.150, 1.741, 1.858, 1.776,
        2.363, 0.919, -0.659, 0.399, 2.983, -0.386, 0.512, 1.663, -0.223],
       [2.151, 0.518, -1.766, 1.487, 1.644, 2.611, 1.509, 0.625, 0.708,
        -0.064, 1.142, 2.312, 1.960, -2.695, 1.258, 1.678, -0.544, 2.857],
       [1.973, 0.213, -1.858, 1.176, 1.765, 2.518, 1.123, 0.456, 0.541,
        -0.198, 1.128, 2.470, 1.878, -2.272, 1.631, 1.641, -0.314, 2.717],
       [1.298, -0.518, -1.628, 0.263, 1.979, 1.996, 0.008, 0.070, 0.148,
        -0.371, 1.064, 2.666, 1.493, -0.511, 2.581, 1.424, 0.564, 2.027],
       [2.470, 1.520, -0.972, 2.325, 1.159, 2.584, 2.565, 1.200, 1.260,
        0.535, 1.160, 1.593, 2.026, -3.263, 0.094, 1.663, -0.941, 2.925],
       [0.043, 1.809, 3.860, 0.849, 0.227, -0.509, 0.907, 1.556, 1.470,
        2.206, 0.873, -0.479, 0.130, 

In [25]:
Z1_relu = np.maximum(0, Z)
Z1_relu

array([[0.948, 0.000, 0.000, 0.000, 1.993, 1.656, 0.000, 0.000, 0.038,
        0.000, 1.028, 2.609, 1.265, 0.470, 2.894, 1.282, 1.025, 1.603],
       [0.528, 2.375, 3.750, 1.530, 0.049, 0.000, 1.741, 1.858, 1.776,
        2.363, 0.919, 0.000, 0.399, 2.983, 0.000, 0.512, 1.663, 0.000],
       [2.151, 0.518, 0.000, 1.487, 1.644, 2.611, 1.509, 0.625, 0.708,
        0.000, 1.142, 2.312, 1.960, 0.000, 1.258, 1.678, 0.000, 2.857],
       [1.973, 0.213, 0.000, 1.176, 1.765, 2.518, 1.123, 0.456, 0.541,
        0.000, 1.128, 2.470, 1.878, 0.000, 1.631, 1.641, 0.000, 2.717],
       [1.298, 0.000, 0.000, 0.263, 1.979, 1.996, 0.008, 0.070, 0.148,
        0.000, 1.064, 2.666, 1.493, 0.000, 2.581, 1.424, 0.564, 2.027],
       [2.470, 1.520, 0.000, 2.325, 1.159, 2.584, 2.565, 1.200, 1.260,
        0.535, 1.160, 1.593, 2.026, 0.000, 0.094, 1.663, 0.000, 2.925],
       [0.043, 1.809, 3.860, 0.849, 0.227, 0.000, 0.907, 1.556, 1.470,
        2.206, 0.873, 0.000, 0.130, 4.233, 0.341, 0.363, 2.293, 0.000],

In [124]:
Z1_relu.shape

(8, 18)

In [28]:
hidden_dim = Z1_relu.shape[1]                
D_out      = layer_norm_x.shape[1]       

# Inisialisasi W2, b2 dengan He Initialization & bias=1
W2 = he_initialization((hidden_dim, D_out))
b2 = np.full((D_out,), 1.0, dtype=float)

# Linear layer-2 (tanpa aktivasi)
Z2 = Z1_relu @ W2 + b2    # shape: (N, D_out)

print("W2 shape:", W2.shape)   # (hidden_dim, D_out)
print("b2 shape:", b2.shape)   # (D_out,)
print("Z2 shape:", Z2.shape)   # (N, D_out)
print("W2:\n", np.round(W2, 4))
print("\nPreview Z2 (tanpa residual & layer norm):\n", np.round(Z2, 4))


W2 shape: (18, 3)
b2 shape: (3,)
Z2 shape: (8, 3)
W2:
 [[0.344 0.310 -0.280]
 [-0.103 0.110 0.325]
 [-0.160 -0.062 -0.369]
 [-0.399 0.271 0.452]
 [-0.024 0.335 0.120]
 [-0.215 0.120 0.513]
 [-0.012 0.521 -0.873]
 [0.274 0.029 -0.100]
 [0.031 -0.662 -0.073]
 [0.119 0.493 -0.173]
 [-0.270 -0.167 0.305]
 [0.110 -0.177 0.171]
 [0.032 0.323 -0.234]
 [-0.109 -0.131 -0.488]
 [0.099 0.087 0.002]
 [-0.078 -0.472 -0.140]
 [-0.114 -0.267 -0.054]
 [0.135 0.629 0.058]]

Preview Z2 (tanpa residual & layer norm):
 [[1.206 2.230 1.919]
 [-0.240 1.473 -2.735]
 [1.054 4.476 1.562]
 [1.157 4.147 1.745]
 [1.294 2.795 2.317]
 [0.782 5.306 0.864]
 [-0.359 0.499 -2.875]
 [1.221 3.903 1.856]]


In [31]:
Z2_out = layer_norm_x + Z2
Z2_out

array([[0.111, 2.005, 3.239],
       [0.473, 2.174, -4.150],
       [1.105, 3.227, 2.760],
       [0.964, 3.031, 3.055],
       [0.439, 2.247, 3.720],
       [1.556, 3.894, 1.502],
       [-0.147, 1.603, -4.191],
       [0.872, 2.891, 3.218]])

In [52]:
def layer_norm(x, eps=1e-5):
    # normalisasi per token (per baris)
    mu  = x.mean(axis=1, keepdims=True)
    var = x.var(axis=1, keepdims=True)
    return (x - mu) / np.sqrt(var + eps), mu.squeeze(), np.sqrt(var + eps).squeeze()

Z2_norm, Z_row_mean, Z_row_std = layer_norm(Z2_out)
Z2_norm

array([[-1.301, 0.171, 1.130],
       [0.364, 1.001, -1.366],
       [-1.383, 0.948, 0.435],
       [-1.414, 0.695, 0.720],
       [-1.264, 0.084, 1.181],
       [-0.683, 1.414, -0.731],
       [0.315, 1.036, -1.352],
       [-1.402, 0.544, 0.859]])

In [53]:
print("Z1 mean:", Z_row_mean)
print("Z1 std :", Z_row_std)

Z1 mean: [1.785 -0.501 2.364 2.350 2.135 2.317 -0.912 2.327]
Z1 std : [1.287 2.672 0.910 0.980 1.342 1.115 2.426 1.037]


In [62]:
N, D = Z2_norm.shape
M = D

W_linear = he_initialization((D, M))
b_linear = np.ones((M,), dtype=float)

y_linear = Z2_norm @ W_linear + b_linear

print("W_linear shape:", W_linear.shape)
print("Y_linear:", y_linear.shape)
print("Preview Y_linear:\n", np.round(y_linear, 3))

W_linear shape: (3, 3)
Y_linear: (8, 3)
Preview Y_linear:
 [[2.036 0.960 1.918]
 [0.805 1.125 -0.340]
 [2.170 1.040 1.185]
 [2.172 1.012 1.473]
 [1.999 0.952 1.978]
 [1.663 1.123 0.115]
 [0.847 1.127 -0.335]
 [2.149 0.996 1.619]]


In [63]:
W_linear

array([[0.240, 0.433, 0.454],
       [1.115, 0.496, 0.263],
       [1.024, 0.388, 1.295]])

## Classification

In [73]:
CLS = y_linear[0]       
M   = CLS.shape[0]

np.random.seed(77)

# Linear untuk [CLS] (He Initialization)
H_cls = 9
W_cls = he_initialization((M, H_cls))
b_cls = np.ones((H_cls,), dtype=float)

cls_linear = CLS @ W_cls + b_cls      
cls_tanh   = np.tanh(cls_linear)      

print("CLS shape:", CLS.shape)
print("W_cls shape:", W_cls.shape)
print("cls_linear:\n", np.round(cls_linear, 2))
print("cls_tanh:\n", np.round(cls_tanh, 2))

CLS shape: (3,)
W_cls shape: (3, 9)
cls_linear:
 [0.690 2.090 0.780 -0.240 -0.570 -3.400 -0.560 0.130 2.250]
cls_tanh:
 [0.600 0.970 0.650 -0.230 -0.510 -1.000 -0.510 0.130 0.980]


In [74]:
CLS

array([2.036, 0.960, 1.918])

In [88]:
W_cls

array([[0.183, 0.540, -0.473, 0.333, 0.381, -1.612, 0.225, -1.287, 1.281],
       [-0.592, 0.250, 0.950, -0.857, -0.797, -0.522, 1.386, -1.050,
        -0.154],
       [-0.060, -0.131, -0.089, -0.569, -0.821, -0.321, -1.746, 1.438,
        -0.632]])

In [84]:
feat = cls_tanh                
F    = feat.shape[0]
num_labels = 3

np.random.seed(1313)

# Bobot classifier (He Initialization) dan bias=1 
W_clf = he_initialization((F, num_labels))
b_clf = np.ones((num_labels,), dtype=float)

logits = feat @ W_clf + b_clf      # (3,)
# Softmax
logits_shift = logits - logits.max()
probs = np.exp(logits_shift) / np.exp(logits_shift).sum()

print("W_clf shape:", W_clf.shape)
print("logits:", np.round(logits, 3))
print("softmax probs (Class A, Class B, Class C):", np.round(probs, 4))

W_clf shape: (9, 3)
logits: [0.780 -0.132 -0.279]
softmax probs (Class A, Class B, Class C): [0.572 0.230 0.198]


In [86]:
W_clf

array([[-0.006, -0.682, 0.279],
       [-0.161, 0.478, 0.224],
       [0.063, -0.305, 0.527],
       [-0.227, 0.210, 0.681],
       [0.471, -0.298, 0.389],
       [0.080, 0.029, 0.292],
       [-0.075, 0.498, 0.533],
       [-0.623, -0.016, -0.310],
       [0.214, -0.826, -1.070]])